# Исследование надежности заемщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели ***кредитного скоринга*** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Описание данных

- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

## Загрузка данных и изучение общей информации

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод:**

В таблице 21525 записей о потенциальных заемщиках. Наблюдаются пропуски в колонках с указанием трудового стажа и ежемесячным доходом.

## Предобработка данных

### Удаление пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполню пропуски в этом столбце медианным значением по каждому типу из столбца `income_type`.

In [5]:
for meaning in data['income_type'].unique():
    data.loc[(data['income_type'] == meaning) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == meaning), 'total_income'].median()

Посмотрю на уникальные значения столбца `days_employed`.

In [6]:
data['days_employed'].unique()

array([-8437.67302776, -4024.80375385, -5623.42261023, ...,
       -2113.3468877 , -3112.4817052 , -1984.50758853])

В данных наблюдаются аномальные значения, в виде отрицательного значения количество дней трудового стажа. Обработаю значения в этом столбце: заменю все отрицательные значения положительными с помощью метода `abs()`.

In [7]:
data['days_employed'] = data['days_employed'].abs()

Выведу для каждого типа занятости медианное значение трудового стажа `days_employed` в днях.

In [8]:
data.groupby('income_type')['days_employed'].median()

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получаются аномально большие значения. Исправить такие значения сложно, поэтому оставлю их как есть. Тем более этот столбец не понадобится в исследовании.

Рассмотрю перечень уникальных значений столбца `children`.

In [10]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце `children` есть два аномальных значения. Удалю строки, в которых встречаются такие аномальные значения из датафрейма data.

In [11]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [12]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

Теперь заполню пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.

In [13]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [14]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод:**
    
- исключены аномальные значения в столбце `children`
- заполнены пропуски в столбцах `total_income` и `days_employed` медианным значением по каждому типу из столбца `income_type`.

### Изменение типов данных

Заменю вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.

In [15]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

В столбце `education` есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведу их к нижнему регистру.

In [16]:
data['education'] = data['education'].str.lower()

Посмотрю на явные дубликаты в данных. Если они имеются, удалю их.

In [18]:
data.duplicated().sum()

71

In [19]:
data = data.drop_duplicates()

### Категоризация данных

Создам в датафрейме `data` столбец `total_income_category` с категориями в следующих диапазонах:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

Используйю собственную функцию с именем `categorize_income()` и метод `apply()`.

In [20]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [21]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Теперь выведу перечень уникальных целей взятия кредита из столбца `purpose`.

In [22]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

В столбце `purpose` наблюдаются повторяющиеся цели, но написаные по-разному. Создам функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [23]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [24]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Вопросы исследования

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для ответа на вопрос:
- сгруппирую значения в столбце `children` и узнаю общее количество значений в столбце задолжностей `debt` в зависимости от количества детей
- сгруппирую значения в столбце `children` и узнаю общее количество задолжностей по возврату кредитов для каждой категории
- найду соотношение между задолжностями по выплате кредитов и количеству детей в процентах
- объединю ячейки в одну сводную таблицу

In [25]:
data.groupby(by=['children'])['debt'].count()

children
0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: debt, dtype: int64

In [26]:
data.groupby(by=['children'])['debt'].sum()

children
0    1063
1     444
2     194
3      27
4       4
5       0
Name: debt, dtype: int64

In [27]:
ratio = (data.groupby(by=['children'])['debt'].sum()) / \
(data.groupby(by=['children'])['debt'].count())*100

In [28]:
print(f'Соотношение задолжностей по возврату кредитов в зависимости от количества детей:\n {ratio}')

Соотношение задолжностей по возврату кредитов в зависимости от количества детей:
 children
0    7.543822
1    9.234609
2    9.454191
3    8.181818
4    9.756098
5    0.000000
Name: debt, dtype: float64


In [29]:
data_children = data.pivot_table(index='children', values='debt', aggfunc=['count', 'sum', 'mean'])
data_children.columns = ['amount_children', 'amount_debt', 'ratio']

In [30]:
data_children['ratio'] = data_children['ratio'].apply(lambda x:f'{x:.2%}')

In [31]:
data_children

,amount_children,amount_debt,ratio
children,,,
0,14091,1063,7.54%
1,4808,444,9.23%
2,2052,194,9.45%
3,330,27,8.18%
4,41,4,9.76%
5,9,0,0.00%


**Вывод:** 

Рассматривая полученные данные, можно сделать вывод, что небольшая зависимость между количеством детей и возвратом кредита все же есть. Так как основные заемщики - клиенты без детей и их процент возврата кредита в срок выше, чем у клиентов с детьми. Также можно сказать, что с увеличением количества детей, клиенты берут кредиты реже, либо банк на первоначальных этапах проверки не одобряет кредиты таким клиентам, и становится сложно определять зависимость из-за маленького количества взятых кредитов, это хорошо видно по категории клиентов у которых 5 детей и взято всего 9 кредитов. 

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для ответа на вопрос:

- сгруппирую значения в столбце `family_status` и узнаю общее количество значений в столбце задолжностей `debt` в зависимости от семейного положения клиентов
- сгруппирую значения в столбце `family_status` и узнаю общее количество задолжностей по возврату кредитов для каждой категории
- найду соотношение между задолжностями по выплате кредитов и семейного положения клиентов в процентах
- объединю ячейки в одну сводную таблицу

In [32]:
data.groupby(by=['family_status'])['debt'].count()

family_status
Не женат / не замужем     2796
в разводе                 1189
вдовец / вдова             951
гражданский брак          4134
женат / замужем          12261
Name: debt, dtype: int64

In [33]:
data.groupby(by=['family_status'])['debt'].sum()

family_status
Не женат / не замужем    273
в разводе                 84
вдовец / вдова            63
гражданский брак         385
женат / замужем          927
Name: debt, dtype: int64

In [34]:
ratio_status = (data.groupby(by=['family_status'])['debt'].sum()) / \
(data.groupby(by=['family_status'])['debt'].count())*100

In [35]:
print(f'Соотношение задолжностей по возврату кредитов в зависимости от семейного положения клиента:\n {ratio_status}')

Соотношение задолжностей по возврату кредитов в зависимости от семейного положения клиента:
 family_status
Не женат / не замужем    9.763948
в разводе                7.064760
вдовец / вдова           6.624606
гражданский брак         9.313014
женат / замужем          7.560558
Name: debt, dtype: float64


In [36]:
data_status = data.pivot_table(index='family_status', values='debt', aggfunc=['count', 'sum', 'mean'])
data_status.columns = ['amount_status', 'amount_debt', 'ratio']

In [37]:
data_status['ratio'] = data_status['ratio'].apply(lambda x:f'{x:.2%}')

In [38]:
data_status

,amount_status,amount_debt,ratio
family_status,,,
Не женат / не замужем,2796,273,9.76%
в разводе,1189,84,7.06%
вдовец / вдова,951,63,6.62%
гражданский брак,4134,385,9.31%
женат / замужем,12261,927,7.56%


**Вывод:** 

Исходя из полученных данных, можно сделать вывод, что клиенты, которые состоят или стостояли когда либо в браке, возвращают кредиты в срок довольно чаще, чем клиенты, которые никогда не были в браке.

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для ответа на вопрос:

- сгруппирую значения в столбце `total_income_category` и узнаю общее количество значений в столбце задолжностей `debt` в зависимости от уровня дохода клиентов
- сгруппирую значения в столбце `total_income_category` и узнаю общее количество задолжностей по возврату кредитов для каждой категории
- найду соотношение между задолжностями по выплате кредитов и уровнем дохода клиентов в процентах
- объединю ячейки в одну сводную таблицу

In [39]:
data.groupby(by=['total_income_category'])['debt'].count()

total_income_category
A       25
B     5014
C    15921
D      349
E       22
Name: debt, dtype: int64

In [40]:
data.groupby(by=['total_income_category'])['debt'].sum()

total_income_category
A       2
B     354
C    1353
D      21
E       2
Name: debt, dtype: int64

In [41]:
ratio_income = (data.groupby(by=['total_income_category'])['debt'].sum()) / \
(data.groupby(by=['total_income_category'])['debt'].count())*100

In [42]:
print(f'Соотношение задолжностей по возврату кредитов в срок в зависимости от уровня дохода клиента:\n {ratio_income}')

Соотношение задолжностей по возврату кредитов в срок в зависимости от уровня дохода клиента:
 total_income_category
A    8.000000
B    7.060231
C    8.498210
D    6.017192
E    9.090909
Name: debt, dtype: float64


In [43]:
data_income = data.pivot_table(index='total_income_category', values='debt', aggfunc=['count', 'sum', 'mean'])
data_income.columns = ['amount_income', 'amount_debt', 'ratio']

In [44]:
data_income['ratio'] = data_income['ratio'].apply(lambda x:f'{x:.2%}')

In [45]:
data_income

,amount_income,amount_debt,ratio
total_income_category,,,
A,25,2,8.00%
B,5014,354,7.06%
C,15921,1353,8.50%
D,349,21,6.02%
E,22,2,9.09%


**Вывод:** 

Глядя на полученные расчеты, зависимость можно рассматривать только по 2 группам с достаточным количеством наблюдений. Основные кредитополучатели из датафрейма соответствуют категогиям "В" и "С". Можно сделать вывод, что категория клиентов с более высоким доходом, реже допускает просрочек по кредиту. Думаю, это связано с тем, что данная категория более платежеспособна.


#### Как разные цели кредита влияют на его возврат в срок?

Для ответа на вопрос:

- сгруппирую значения в столбце `purpose_category` и узнаю общее количество значений в столбце задолжностей `debt` в зависимости от целей кредита клиентов
- сгруппирую значения в столбце `purpose_category` и узнаю общее количество задолжностей по возврату кредитов для каждой категории
- найду соотношение между задолжностями по выплате кредитов и целей кредита клиентов в процентах
- объединю ячейки в одну сводную таблицу

In [46]:
data.groupby(by=['purpose_category'])['debt'].count()

purpose_category
операции с автомобилем       4279
операции с недвижимостью    10751
получение образования        3988
проведение свадьбы           2313
Name: debt, dtype: int64

In [47]:
data.groupby(by=['purpose_category'])['debt'].sum()

purpose_category
операции с автомобилем      400
операции с недвижимостью    780
получение образования       369
проведение свадьбы          183
Name: debt, dtype: int64

In [48]:
ratio_purpose = (data.groupby(by=['purpose_category'])['debt'].sum()) / \
(data.groupby(by=['purpose_category'])['debt'].count())*100

In [49]:
print(f'Соотношение задолжностей по возврату кредитов в срок в зависимости от целей кредита:\n {ratio_purpose}')

Соотношение задолжностей по возврату кредитов в срок в зависимости от целей кредита:
 purpose_category
операции с автомобилем      9.347978
операции с недвижимостью    7.255139
получение образования       9.252758
проведение свадьбы          7.911803
Name: debt, dtype: float64


In [50]:
data_purpose = data.pivot_table(index='purpose_category', values='debt', aggfunc=['count', 'sum', 'mean'])
data_purpose.columns = ['amount_income', 'amount_debt', 'ratio']

In [51]:
data_purpose['ratio'] = data_purpose['ratio'].apply(lambda x:f'{x:.2%}')

In [52]:
data_purpose

,amount_income,amount_debt,ratio
purpose_category,,,
операции с автомобилем,4279,400,9.35%
операции с недвижимостью,10751,780,7.26%
получение образования,3988,369,9.25%
проведение свадьбы,2313,183,7.91%


**Вывод:** 

Самая популярная цель кредита - это недвижимость, а также у данной категории, самый высокий процент возврата кредита - 7,9%, это обусловлено тем, что чаще всего данный кредит имеет высокую стоимость и в случае задержек начисляется высокая по сумме пеня, я так же люди боятся потерять недвижимость. 

Также низкий процент задержек по выплате кредита на проведение свадеб, т.к. ранее был сделан вывод, что клиенты после заключения брака относятся к выплате кредита более ответственно. 

Кредиты на операции с автомобилем и получения образования имет более высокий и схожий процент по задержке возврата кредитов, думаю, это обусловлено тем, что данные категории операций имеют более низкий приоритет важности, в отличие от операций с недвижимостью. Автомобиль после покупки только теряет в цене и частая практика менять автомобиль раз в 3-5 лет, поэтому клиенты в меньшей степени боятся просрочек или, то что банк заберет автомобиль. Приоритет выплат по операциям связанных с получением образования снижается после получения образования, так как цель клиента достигнута и в основном данная категория кредитов беззалоговая.

#### Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* 

Технологические. Возникают во время копирования, считывания или записи данных, изменения формата файла.


Вызванные человеческим фактором. Люди постоянно ошибаются. 

#### Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* 

Среднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства. 
В случае отклонения распределения в выборке слишком большим или слишком малым значением среднее значение использовать некорректно, так как оно является слишком нехарактерным для изучаемой выборки.
Данный вопрос можно рассмотреть на примере уровня дохода клиентов банка. Минимальное значение дохода одного из клиентов около 20 тыс. рублей, а максимальное значение дохода одного из клиентов более 2 млн рублей. При том, что основная категория клиентов банка имеет доход от 50 тыс. до 200 тыс. рублей. Когда есть оторванные от основной массы лидеры, их результаты сильно смещают значение среднего вверх. В следствии чего оценивать доходы клиентов лучше медианой, она предоставляет более объективную информацию о доходах клиентов банка, чем среднее значение. 

### Общий вывод

Основной задачей проекта было разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Для того, чтобы результаты исследования учтесть при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.
После проведенного анализа были выявлены основные пункты зависимости возврата клиентами кредита в срок.
К данным пунктам относятся:
- клиенты не имеющие детей;
- клиенты побывавшие в браке;
- клиенты с доходом от 200001 до 1000000 рублей;
- цель кредита - операции с недвижимостью.

Полученный результат анализа данных соответствует поставленной задаче.